In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

# 데이터 불러오기
file_path = "data/generated_meal_plan.csv"
df = pd.read_csv(file_path)

# 필요 없는 열 제거 (아침, 점심, 저녁 음식명은 학습에 사용하지 않음)
df = df.drop(columns=["아침", "점심", "저녁"])

# 범주형 변수 변환 (성별, 체형, 목표 → 숫자로 인코딩)
label_encoders = {}
for col in ["성별", "체형", "목표"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # 인코더 저장 (추후 예측할 때 필요)

# 입력(X)과 출력(y) 분리
X = df[["성별", "체형", "목표"]]
y = df[["총 칼로리", "탄수화물", "단백질", "지방"]]

# 데이터 분할 (학습용 80%, 검증용 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.fillna(X_train.mean())  # 평균값으로 대체
X_test = X_test.fillna(X_test.mean())
y_train = y_train.fillna(y_train.mean())
y_test = y_test.fillna(y_test.mean())

# XGBoost 모델 학습
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=300, learning_rate=0.01, max_depth=3, subsample=0.8)
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 평가 (평균 절대 오차)
mae = mean_absolute_error(y_test, y_pred)
print(f"모델 학습 완료! 평균 절대 오차 (MAE): {mae:.2f}")

# 모델 저장
model.save_model("data/xgboost_meal_plan.json")
print("모델 저장 완료! 파일: xgboost_meal_plan.json")


✅ 모델 학습 완료! 평균 절대 오차 (MAE): 240.83
✅ 모델 저장 완료! 파일: xgboost_meal_plan.json


In [2]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 예측값 생성
y_pred = model.predict(X_test)

# 평가 지표 계산
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 결과 출력
print(f"모델 평가 결과")
print(f"평균 제곱 오차 (MSE): {mse:.2f}")
print(f"평균 제곱근 오차 (RMSE): {rmse:.2f}")
print(f"결정계수 (R² Score): {r2:.4f}")


📊 모델 평가 결과
🔹 평균 제곱 오차 (MSE): 334332.84
🔹 평균 제곱근 오차 (RMSE): 578.22
🔹 결정계수 (R² Score): -0.3327


In [3]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# XGBoost 모델 생성
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# 하이퍼파라미터 검색 공간 정의
param_distributions = {
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
    "max_depth": [3, 5, 7, 9, 12],
    "n_estimators": [100, 200, 300, 400, 500],
    "subsample": [0.5, 0.7, 0.8, 0.9, 1],
    "colsample_bytree": [0.5, 0.7, 0.8, 0.9, 1]
}

# 랜덤 서치 실행 (n_iter는 탐색 횟수)
random_search = RandomizedSearchCV(
    xgb_model, param_distributions, 
    n_iter=20, scoring="neg_mean_absolute_error", 
    cv=5, verbose=1, random_state=42, n_jobs=-1
)

# 학습 수행
random_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("최적 하이퍼파라미터:", random_search.best_params_)

# 최적 모델로 테스트 데이터 평가
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"📊 모델 평가 결과")
print(f"🔹 MSE: {mse:.2f}")
print(f"🔹 RMSE: {rmse:.2f}")
print(f"🔹 R² Score: {r2:.4f}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
최적 하이퍼파라미터: {'subsample': 0.5, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.5}
📊 모델 평가 결과
🔹 MSE: 316368.54
🔹 RMSE: 562.47
🔹 R² Score: -0.1298


In [10]:
from sklearn.model_selection import GridSearchCV

# 더 정밀한 하이퍼파라미터 범위 설정
param_grid = {
    "learning_rate": [0.03, 0.05, 0.07],
    "max_depth": [2, 3, 4],
    "n_estimators": [150, 200, 250],
    "subsample": [0.4, 0.5, 0.6],
    "colsample_bytree": [0.4, 0.5, 0.6]
}

# Grid Search 실행
grid_search = GridSearchCV(
    xgb.XGBRegressor(objective="reg:squarederror", random_state=42),
    param_grid,
    scoring="neg_mean_absolute_error",
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 학습 수행
grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("🔎 최적 하이퍼파라미터:", grid_search.best_params_)

# 최적 모델로 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"📊 모델 평가 결과")
print(f"🔹 MSE: {mse:.2f}")
print(f"🔹 RMSE: {rmse:.2f}")
print(f"🔹 R² Score: {r2:.4f}")


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
🔎 최적 하이퍼파라미터: {'colsample_bytree': 0.4, 'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 150, 'subsample': 0.6}
📊 모델 평가 결과
🔹 MSE: 286115.56
🔹 RMSE: 534.90
🔹 R² Score: -0.0459


In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)